In [1]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
import os
import random

import cv2
import keras.utils
import numpy as np
from keras.layers import Input, Dense,Flatten, Concatenate
from keras.models import Model
from keras.ops import empty

keras.utils.set_random_seed(42)
np.random.seed(42)
random.seed(42)


###TRAINING VARIABLES

sample_count = 79999
epoch_count = 100
batch_size = 7
validation_count = 20000
initial_learning_rate=.001
decay_steps=8888
decay_rate=0.97
dataset_name = "SuToBoCM"



inp1 = Input(shape=(320, 240, 1))
inp2 = Input(shape=(320, 240, 1))
empty = keras.ops.empty(shape=(batch_size, 320, 240, 1))

concat = Concatenate(axis=-1)([inp1, inp2])
concat = Concatenate(axis=-1)([concat, empty])


resnet = keras.applications.ResNet50V2(
    include_top=False,
    weights=None,
    input_tensor=concat,
    input_shape=(320, 240, 3),
    pooling=None,
    classes=1000,
    classifier_activation="relu",
)

flat = Flatten()(resnet.output)
dense = Dense(128, activation="relu")(flat)

chest_out = Dense(1, activation='linear', name='chest_out')(dense)
waist_out = Dense(1, activation='linear', name='waist_out')(dense)
pelvis_out = Dense(1, activation='linear', name='pelvis_out')(dense)
# neck_out = Dense(1, activation='linear', name='neck_out')(dense)
bicep_out = Dense(1, activation='linear', name='bicep_out')(dense)
thigh_out = Dense(1, activation='linear', name='thigh_out')(dense)
# knee_out = Dense(1, activation='linear', name='knee_out')(dense)
shoulder_to_wrist_out = Dense(1, activation='linear', name='shoulder_to_wrist_out')(dense)
leg_out = Dense(1, activation='linear', name='leg_out')(dense)
calf_out = Dense(1, activation='linear', name='calf_out')(dense)
# head_out = Dense(1, activation='linear', name='head_out')(dense)
wrist_out = Dense(1, activation='linear', name='wrist_out')(dense)
arm_out = Dense(1, activation='linear', name='arm_out')(dense)
shoulder_to_shoulder_out = Dense(1, activation='linear', name='shoulder_to_shoulder_out')(dense)
# torso_out = Dense(1, activation='linear', name='torso_out')(dense)
inner_leg_out = Dense(1, activation='linear', name='inner_leg_out')(dense)


model = Model(
    inputs=[inp1, inp2],
    outputs=[
        chest_out,
        waist_out,
        pelvis_out,
        # neck_out,
        bicep_out,
        thigh_out,
        # knee_out,
        shoulder_to_wrist_out,
        leg_out,
        calf_out,
        # head_out,
        wrist_out,
        arm_out,
        shoulder_to_shoulder_out,
        # torso_out,
        inner_leg_out
    ],
    name='ResNet50V2'
)




folder_path = f"{dataset_name} - {sample_count}s{epoch_count}e{validation_count}v {"" if initial_learning_rate == 0.001 else initial_learning_rate + "ilr"}{decay_steps}s{decay_rate}d"

try:
    os.mkdir(os.path.join("./logs/" + folder_path))
except OSError as error:
    print(error)
    

#model.save_weights("default.weights.h5")

# model.summary()
# 
from keras.utils import plot_model

plot_model(model, to_file='ResNet50+Side.png', show_shapes=True, show_layer_names=True, show_layer_activations=True)

In [ ]:
class Database_Loader(keras.utils.Sequence):

    def __init__(self, image_location, data_location, sample_count, batch_size, dataset = "Surreact", shuffle=True, seed=0, input_dimensions=(240, 320), prefix="Avatar_", use_memory=False, load_data = True, random_sample = False):
        self.image_location = image_location
        self.data_location = data_location
        self.sample_count = sample_count
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seed = seed
        self.input_dimensions = input_dimensions
        self.prefix = prefix
        self.use_memory = use_memory
        self.load_data = load_data
        self.random_sample = random_sample
        self.dataset = dataset

        super().__init__(workers=2, use_multiprocessing=True)

        if self.random_sample:
            self.IDs = random.sample(range(len(os.listdir(self.image_location))),self.sample_count)
        else:
            self.IDs = [x for x in range(self.sample_count)]
            self._load_data()

        super().__init__(workers=1, use_multiprocessing=False)
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.sample_count / self.batch_size))


    def _load_data(self):
        if self.load_data:
            self.data = dict()
            for index in self.IDs:
                self.data[index] = np.load(os.path.join(self.data_location + self.prefix + f"{index:06d}.npy"))

        if self.use_memory:
            self.images = dict()
            for index in self.IDs:
                self.images[index] = cv2.imread(os.path.join(self.image_location + self.prefix + f"{index:06d}.png"), cv2.IMREAD_GRAYSCALE)

    def __getitem__(self, index):
        X1 = np.empty(shape=(self.batch_size, 320, 240))
        X2 = np.empty(shape=(self.batch_size, 320, 240))
        y = {
            'chest_out': [],
            'waist_out': [],
            'pelvis_out': [],
            # 'neck_out': [],
            'bicep_out': [],
            'thigh_out': [],
            # 'knee_out': [],
            'shoulder_to_wrist_out': [],
            'leg_out': [],
            'calf_out': [],
            # 'head_out': [],
            'wrist_out': [],
            'arm_out': [],
            'shoulder_to_shoulder_out': [],
            # 'torso_out': [],
            'inner_leg_out': [],
        }

        start_index = index * self.batch_size + 1
        for i in range(self.batch_size):
            name = f"{self.IDs[(start_index + i)%self.sample_count]:06d}"


            if self.load_data:
                current_measurement = self.data[self.IDs[(start_index + i)%self.sample_count]][:-1]
            else:
                current_measurement = np.load(os.path.join(self.data_location + self.prefix + name + ".npy"))[:-1]

            if self.use_memory:
                X1[i,] = self.images[self.IDs[(start_index + i)%self.sample_count]]

            else:
                X1[i,] = cv2.imread(os.path.join(self.image_location + self.prefix + name + ".png"), cv2.IMREAD_GRAYSCALE)
                X2[i,] = cv2.imread(os.path.join(self.image_location + "../images_side/" + self.prefix + name + ".png"), cv2.IMREAD_GRAYSCALE)

            if self.dataset == "Surreact":
                y['chest_out'].append([current_measurement[0]])
                y['waist_out'].append([current_measurement[1]])
                y['pelvis_out'].append([current_measurement[2]])
                # y['neck_out'].append([current_measurement[3]])
                y['bicep_out'].append([current_measurement[4]])
                y['thigh_out'].append([current_measurement[5]])
                # y['knee_out'].append([current_measurement[6]])
                y['shoulder_to_wrist_out'].append([current_measurement[7]])
                y['leg_out'].append([current_measurement[8]])
                y['calf_out'].append([current_measurement[9]])
                # y['head_out'].append([current_measurement[10]])
                y['wrist_out'].append([current_measurement[11]])
                y['arm_out'].append([current_measurement[12]])
                y['shoulder_to_shoulder_out'].append([current_measurement[13]])
                # y['torso_out'].append([current_measurement[14]])
                y['inner_leg_out'].append([current_measurement[15]])

        for key, value in y.items():
            y[key] = np.array(value)

        return [X1, X2], y

    def on_epoch_end(self):
        if self.random_sample:
            self.IDs = random.sample(range(len(os.listdir(self.image_location))),self.sample_count)
            self._load_data()

        if self.shuffle:
            np.random.seed(self.IDs[0]+self.seed)
            np.random.shuffle(self.IDs)
        else:
            self.IDs = np.arange(self.sample_count)


In [ ]:
quickTrain = {
                'image_location': "./Surreact-APose/train/images_front/", 
                'data_location': "./Surreact-APose/train/measurementsCM/", 
                'sample_count': sample_count, 
                'batch_size': batch_size, 
                'seed':  69,#np.random.randint(0, 10000), 
                'use_memory': False,
                'random_sample': False,
                'shuffle': True}

quickValidate = {
                'image_location': "./Surreact-APose/test/images_front/",
                'data_location': "./Surreact-APose/test/measurementsCM/",
                'sample_count': validation_count,
                'batch_size': batch_size,
                'seed': 69, #np.random.randint(0, 10000),
                'use_memory': False,
                'random_sample': False}

train_generator = Database_Loader(**quickTrain)
validation_generator = Database_Loader(**quickValidate)

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, EarlyStopping
from keras.losses import MeanSquaredError


lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

stop_no_improvement = keras.callbacks.EarlyStopping(monitor="loss", patience=3)


model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss={
        'chest_out': MeanSquaredError(),
        'waist_out': MeanSquaredError(),
        'pelvis_out': MeanSquaredError(),
        # 'neck_out': MeanSquaredError(),
        'bicep_out': MeanSquaredError(),
        'thigh_out': MeanSquaredError(),
        # 'knee_out': MeanSquaredError(),
        'shoulder_to_wrist_out': MeanSquaredError(),
        'leg_out': MeanSquaredError(),
        'calf_out': MeanSquaredError(),
        # 'head_out': MeanSquaredError(),
        'wrist_out': MeanSquaredError(),
        # 'arm_out': MeanSquaredError(),
        'shoulder_to_shoulder_out': MeanSquaredError(),
        # 'torso_out': MeanSquaredError(),
        # 'inner_leg_out': MeanSquaredError()
    },
    metrics={
        'chest_out': ['mae'],
        'waist_out': ['mae'],
        'pelvis_out': ['mae'],
        # 'neck_out': ['mae'],
        'bicep_out': ['mae'],
        'thigh_out': ['mae'],
        # 'knee_out': ['mae'],
        'shoulder_to_wrist_out': ['mae'],
        'leg_out': ['mae'],
        'calf_out': ['mae'],
        # 'head_out': ['mae'],
        'wrist_out': ['mae'],
        # 'arm_out': ['mae'],
        'shoulder_to_shoulder_out': ['mae'],
        # 'torso_out': ['mae'],
        # 'inner_leg_out': ['mae'],
    },
)



checkpoint_filepath = './models/ResNet50V2.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True)

model.fit(
    x=train_generator,
    validation_data=validation_generator,
    callbacks=[
        
        TensorBoard(write_graph=True,log_dir="./logs/"+folder_path),
        model_checkpoint_callback,
    ],
    batch_size=batch_size,
    epochs=epoch_count,
    verbose=1,
)

model.save("./models/full/ResNet50V2.keras")
# 
# for i in range(10000):
#     image = cv2.imread(f"../Surreact-APose/train/imgs_nobg_frontEDITED/Avatar_{6969+i:06d}.png", cv2.IMREAD_GRAYSCALE)
# 
#     print(model.predict(np.array([image])))
#     